In [1]:
! mlflow --version

mlflow, version 2.13.1


In [2]:
import sys
import os
sys.path.append('/workspaces/mlop_zoomcamp_learning/utils')
import preprocess_data
import mlflow

In [3]:
x=list(sys.path)

In [4]:
for i in range(0,len(x)):
    print(x[i])

/home/codespace/anaconda3/envs/mlflow_env/lib/python39.zip
/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9
/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/lib-dynload

/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages
/workspaces/mlop_zoomcamp_learning/utils


In [5]:
df_jan=preprocess_data.read_dataframe('../data/green_tripdata_2023-01.parquet')

In [6]:
df_jan.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,...,0.5,4.03,0.0,None,1.0,24.18,1.0,1.0,2.75,11.016667
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,...,0.5,2.64,0.0,None,1.0,15.84,1.0,1.0,0.00,6.766667
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,...,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00,6.333333
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,...,1.5,1.70,0.0,None,1.0,10.20,1.0,1.0,0.00,5.816667
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,...,1.5,0.00,0.0,None,1.0,8.00,1.0,1.0,0.00,5.966667


In [7]:
! pwd

/workspaces/mlop_zoomcamp_learning/2-exp-tracking


In [8]:
!python ../utils/preprocess_data.py --raw_data_path ../data/ --dest_path ./output

In [15]:
!python ../utils/train.py --data_path ./output

2024/06/06 02:58:25 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/06/06 02:58:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/06/06 02:58:25 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/06/06 02:58:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'r

In [22]:
#made a mistake in experiment 'random-forest-hyperopt'
# to delete 
def delete_experiment_by_name(experiment_name):
    # Get the experiment ID by name
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is None:
        print(f"Experiment with name '{experiment_name}' does not exist.")
        return
    
    experiment_id = experiment.experiment_id

    # Delete the experiment by ID
    mlflow.delete_experiment(experiment_id)
    print(f"Experiment with ID '{experiment_id}' and name '{experiment_name}' has been marked as deleted.")

# Replace 'your_experiment_name' with the name of the experiment you want to delete
delete_experiment_by_name('random-forest-hyperopt')

MlflowException: No Experiment with id=2 exists

In [59]:
mlflow.search_experiments() 

[<Experiment: artifact_location='/workspaces/mlop_zoomcamp_learning/2-exp-tracking/mlruns/4', creation_time=1717650702391, experiment_id='4', last_update_time=1717650702391, lifecycle_stage='active', name='random-forest-hyperopt-2', tags={}>,
 <Experiment: artifact_location='/workspaces/mlop_zoomcamp_learning/2-exp-tracking/artifacts/3', creation_time=1717644644066, experiment_id='3', last_update_time=1717644644066, lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='/workspaces/mlop_zoomcamp_learning/2-exp-tracking/mlruns/1', creation_time=1717640080544, experiment_id='1', last_update_time=1717640080544, lifecycle_stage='active', name='hw2-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlop_zoomcamp_learning/2-exp-tracking/artifacts/0', creation_time=1717639786032, experiment_id='0', last_update_time=1717639786032, lifecycle_stage='active', name='Default', tags={}>]

In [37]:
import os
import shutil
import mlflow
from sqlalchemy import create_engine, text

def delete_experiment_permanently(experiment_name, db_uri, artifact_root):
    # Get the experiment details by name
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is None:
        print(f"Experiment with name '{experiment_name}' does not exist.")
        return

    experiment_id = experiment.experiment_id

    # Delete the experiment metadata from the backend store
    engine = create_engine(db_uri)
    with engine.connect() as connection:
        connection.execute(text("DELETE FROM metrics WHERE run_uuid IN (SELECT run_uuid FROM runs WHERE experiment_id = :experiment_id)"), {"experiment_id": experiment_id})
        connection.execute(text("DELETE FROM params WHERE run_uuid IN (SELECT run_uuid FROM runs WHERE experiment_id = :experiment_id)"), {"experiment_id": experiment_id})
        connection.execute(text("DELETE FROM tags WHERE run_uuid IN (SELECT run_uuid FROM runs WHERE experiment_id = :experiment_id)"), {"experiment_id": experiment_id})
        connection.execute(text("DELETE FROM runs WHERE experiment_id = :experiment_id"), {"experiment_id": experiment_id})
        connection.execute(text("DELETE FROM experiments WHERE experiment_id = :experiment_id"), {"experiment_id": experiment_id})

    # Delete the associated artifacts
    artifact_path = os.path.join(artifact_root, str(experiment_id))
    if os.path.exists(artifact_path):
        shutil.rmtree(artifact_path)

    print(f"Experiment with ID '{experiment_id}' and name '{experiment_name}' has been permanently deleted.")

# Replace with your details
db_uri = 'sqlite:///mlflow.db'  # Example for SQLite
artifact_root = './mlruns'
experiment_name = 'random-forest-hyperopt'

delete_experiment_permanently(experiment_name, db_uri, artifact_root)


Experiment with ID '2' and name 'random-forest-hyperopt' has been permanently deleted.


In [40]:
!python ../utils/hpo.py --data_path ./output --num_trials 15

2024/06/06 05:11:42 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt-2' does not exist. Creating a new experiment.
  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋          | 1/15 [00:08<01:58,  8.44s/trial, best loss: 5.370086069268862]/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 13%|█▍         | 2/15 [00:08<00:48,  3.75s/trial, best loss: 5.370086069268862]/home/codespace/anaconda3/envs/mlflow_

In [43]:
#As of mlflow 1.11.0, the recommended way to permanently delete runs within an experiment is
#!mlflow gc --experiment-ids 4

In [57]:
!python ../utils/register_model.py --data_path ./output --top_n 5

/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/mlflow_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editab

In [51]:
model_uri = f"runs:/90a6cb82264e4a7ab4d91bb5ff33e470/models/models"
mlflow.register_model(model_uri=model_uri, name='bestmodel_test')

Registered model 'bestmodel_test' already exists. Creating a new version of this model...
Created version '1' of model 'bestmodel_test'.


<ModelVersion: aliases=[], creation_timestamp=1717653047001, current_stage='None', description=None, last_updated_timestamp=1717653047001, name='bestmodel_test', run_id='90a6cb82264e4a7ab4d91bb5ff33e470', run_link=None, source='/workspaces/mlop_zoomcamp_learning/2-exp-tracking/artifacts/3/90a6cb82264e4a7ab4d91bb5ff33e470/artifacts/models/models', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [53]:
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
experiment = client.get_experiment_by_name('random-forest-best-models')
best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=2,
        order_by=["metrics.test_rmse ASC"]
    )[0]

In [58]:
best_run

<Run: data=<RunData: metrics={'test_rmse': 5.567408012462019, 'val_rmse': 5.335419588556921}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "90a6cb82264e4a7ab4d91bb5ff33e470", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2024-06-06 05:45:03.466769", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.19", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.0", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             

In [56]:
best_run.info.run_id

'90a6cb82264e4a7ab4d91bb5ff33e470'

In [2]:
#first run in terminal
#! mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts

In [9]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("hw2-experiment")

2024/06/06 02:14:40 INFO mlflow.tracking.fluent: Experiment with name 'hw2-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlop_zoomcamp_learning/2-exp-tracking/mlruns/1', creation_time=1717640080544, experiment_id='1', last_update_time=1717640080544, lifecycle_stage='active', name='hw2-experiment', tags={}>

<Experiment: artifact_location='/workspaces/mlop_zoomcamp_learning/02-exp-tracking/mlruns/1', creation_time=1717638502196, experiment_id='1', last_update_time=1717638502196, lifecycle_stage='active', name='hw2-experiment', tags={}>